# Margins for SNP
-[ ] Build / Extract the chains    
-[ ] Get undPrices   
-[ ] Integrate undPrices to the chains   

-[ ] Get the lots   
-[ ] Get margins for with qty as the lot   
-[ ] Make logic for margins from SNP rules   
-[ ] For margins with NaN, replace margins with the one derived from logic   
-[ ] Integrate margins to chains

In [1]:
## THIS CELL SHOULD BE IN ALL VSCODE NOTEBOOKS ##

MARKET = 'SNP'

import pandas as pd
pd.options.display.max_columns=None

# Add `src` to _src.pth in .venv to allow imports in VS Code
from sysconfig import get_path
from pathlib import Path
if 'src' not in Path.cwd().parts:
    src_path = str(Path(get_path('purelib')) / '_src.pth')
    with open(src_path, 'w') as f:
        f.write(str(Path.cwd() / 'src\n'))

# Start the Jupyter loop
from ib_insync import util, IB
util.startLoop()

In [2]:
# Set the root
from from_root import from_root
ROOT = from_root()

from utils import Vars
_vars = Vars(MARKET)
PORT = _vars.PORT
PAPER = _vars.PAPER 
OPT_COLS = _vars.OPT_COLS[0]
DATAPATH = ROOT / 'data' / MARKET.lower()

# Suppress Errors
util.logToFile(DATAPATH.parent.parent / 'log' / 'ztest.log')

# Putting it all together

In [3]:
from utils import make_chains_with_margins, get_strike_closest_to_und
df_chains = make_chains_with_margins(MARKET=MARKET)
df = get_strike_closest_to_und(df_chains)

Getting margins:100%|█████████████████████████| 243/243 [00:13<00:00, 18.44it/s]


In [6]:
symbol = 'AAL'
df_chains[df_chains.symbol == symbol]

,symbol,expiry,strike,right,undId,dte,multiplier,localSymbol,undPrice,iv,sigma,strike_sdev,exchange,lot,margin,comm
23421,AAL,20240315,1.0,P,139673266,1.681179,100,AAL,14.03,0.360961,0.343699,-37.911069,SNP,1.0,1083.8555,1.199784
23422,AAL,20240315,2.0,P,139673266,1.681179,100,AAL,14.03,0.360961,0.343699,-35.001547,SNP,1.0,1083.8555,1.199784
23423,AAL,20240315,2.5,P,139673266,1.681179,100,AAL,14.03,0.360961,0.343699,-33.546786,SNP,1.0,1083.8555,1.199784
23424,AAL,20240315,3.0,P,139673266,1.681179,100,AAL,14.03,0.360961,0.343699,-32.092025,SNP,1.0,1083.8555,1.199784
23425,AAL,20240315,4.0,P,139673266,1.681179,100,AAL,14.03,0.360961,0.343699,-29.182503,SNP,1.0,1083.8555,1.199784
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24442,AAL,20261218,30.0,C,139673266,1009.681179,100,AAL,14.03,0.360961,8.422936,1.896014,SNP,1.0,1083.8555,1.199784
24443,AAL,20261218,32.0,C,139673266,1009.681179,100,AAL,14.03,0.360961,8.422936,2.133460,SNP,1.0,1083.8555,1.199784
24444,AAL,20261218,35.0,C,139673266,1009.681179,100,AAL,14.03,0.360961,8.422936,2.489631,SNP,1.0,1083.8555,1.199784
24445,AAL,20261218,37.0,C,139673266,1009.681179,100,AAL,14.03,0.360961,8.422936,2.727078,SNP,1.0,1083.8555,1.199784


In [7]:
df[df.symbol == symbol]

,symbol,dte,expiry,strike,right,undId,multiplier,localSymbol,undPrice,iv,sigma,strike_sdev,exchange,lot,margin,comm,strk_near_und
0,AAL,1.681179,20240315,14.0,P,139673266,100,AAL,14.03,0.360961,0.343699,-0.087286,SNP,1.0,1083.8555,1.199784,[14.0]


## Get underlying contracts and prices

## Build / Extract the chains
 - Integrates `undPrices` and `undIV` to chains
 - Recalculates `sigma` and `strike_sdev`

In [ ]:
# imports
import asyncio
from utils import get_chains


In [ ]:
df_chains = get_chains(MARKET=MARKET)

In [ ]:
df_temp = df_chains # !!! TEMPORARY STORAGE

In [ ]:
df = get_strike_closest_to_und(df_chains)

### Correct `get_strike_closest_to_und`

In [ ]:
from utils import get_closest_values
how_many = -1

In [ ]:
df1 = df_chains.loc[df_chains.groupby(['symbol', 'strike', 'right']).dte.idxmin()]\
                        .reset_index(drop=True)

strk_near_und = df1[['symbol', 'strike', 'undPrice']].groupby('symbol')\
                                .apply(lambda x: get_closest_values(x.strike, 
                                                                    x.undPrice.min(), 
                                                                    how_many=-1))
strk_near_und.name = 'strk_near_und'
df_ch1 = df_chains.set_index(['symbol', 'dte']).join(strk_near_und)
df_ch = df_ch1[df_ch1.apply(lambda x: x.strike in x.strk_near_und, axis=1)] \
                        .reset_index()

# for SNP limit to lowest dte
if set(df_chains.exchange.to_numpy()).pop() == 'SNP':
    df_ch = df_ch.loc[df_ch.groupby('symbol').dte.idxmin()].reset_index(drop=True)

In [ ]:
df_ch

In [ ]:
df_ch[df_ch.symbol == 'BKNG']

## Get chains with strike price closest to underlying

In [ ]:
# imports
from utils import get_strike_closest_to_und

In [ ]:
df_ch1 = get_strike_closest_to_und(df_chains)

In [ ]:
df_ch1

## Get margins closest to underlying with qty as the lot

In [ ]:
# imports
from utils import opt_margins_with_lot_check
import numpy as np


In [ ]:
# Get underlying margins of chains
df_chund_margins = opt_margins_with_lot_check(df_ch1)

## Integrate with maximum margin, wherever possible

In [ ]:
from utils import join_my_df_with_another

# update df_ch_unds with latest margin and commission
opt_cols = ['symbol', 'expiry', 'strike', 'right']
margin_comm = ['margin', 'comm']
df_ch2 = join_my_df_with_another(df_ch1, df_chund_margins[opt_cols+margin_comm])

# second run
df_rem = df_ch2[df_ch2.margin.isnull()]


# replace df_ch2 with new margin and comm
if not df_rem.empty:
    dfr = opt_margins_with_lot_check(df_rem, multiply_lot=False)


In [ ]:
dfr1 = dfr.set_index(opt_cols).dropna(subset=['margin'])
df_ch2.set_index(opt_cols).update(dfr1, overwrite=False)

In [ ]:
df_ch_all = join_my_df_with_another(df_chains, df_ch2[opt_cols+margin_comm], idx=opt_cols).reset_index()


In [ ]:
# fill missing commissions with max per symbol
commissions = df_ch2.groupby('symbol').comm.max().to_dict()
df_ch_all.comm = df_ch_all.comm.fillna(df_ch_all.symbol.map(commissions))

# fill remaining commissions
df_ch_all.comm = df_ch_all.comm.fillna(max(commissions.values()))

# fill margins
mgn_dict = df_ch_all.groupby('symbol').margin.max().to_dict()
cond = df_ch_all.margin.isnull()
df_ch_all.loc[cond, 'margin'] = df_ch_all[cond].symbol.map(mgn_dict)

# make zero margin as nan
zero_margin_condition = df_ch_all.margin == 0
df_ch_all.loc[zero_margin_condition, 'margin'] = np.nan

## Make logic for margins from SNP rules
### Replace `nan` margins with the one derived from logic

In [ ]:
from utils import compute_snp_df_margins

df = compute_snp_df_margins(df_ch_all)

In [ ]:
df